In [1]:
%%capture
import os
import ipywidgets as widgets
from IPython.display import Javascript

os.environ['R_HOME'] = 'C:\Program Files\R\R-4.4.2'

In [2]:
# !python -m ipykernel install --user --name=ericapp --display-name "Python3(ericapp)"

In [3]:
# Create the HTML widget with styling
title_widget = widgets.HTML(
    value="""
    <div style="
        background-color: #D3D3D3;  /* Light Grey */
        padding: 10px;
        border-radius: 5px;
        text-align: felx-start;
        font-size: 16px;
        font-weight: bold;
    ">
        Occupancy – Spatial Power Analysis
    </div>
    """
)


In [4]:
# Create the HTML widget with a matte blue background
add_info_widget = widgets.HTML(
    value="""
    <div style="
        background-color: #4682B4;  /* Matte Steel Blue */
        padding: 10px;
        border-radius: 5px;
        text-align: flex-start;
        font-size: 16px;
        color: white;  /* White text for contrast */
        height: 150px;  /* Set height */
        margin-right: 15px;
    ">
        Some Additional Information
    </div>
    """
)

In [5]:
# Label widget with grey background
show_label = widgets.HTML(
    value="""
    <div style="
        background-color: #D3D3D3;  /* Light Grey */
        padding: 5px;
        border-radius: 3px;
        font-weight: bold;
        text-align: center;
        width: 50px;
    ">
        Show
    </div>
    """
)

In [6]:
# Checkbox widgets
northbound = widgets.Checkbox(value=False, description="Northbound")
southbound = widgets.Checkbox(value=False, description="Southbound")
eastbound = widgets.Checkbox(value=False, description="Eastbound")
westbound = widgets.Checkbox(value=False, description="Westbound")

# Arrange widgets vertically
checkbox_group = widgets.VBox([show_label, northbound, southbound, eastbound, westbound],
                                layout=widgets.Layout(
                                    align_items="flex-start"
                                )
                             )

In [7]:
# import ipywidgets as widgets
# from IPython.display import display

# # Create a Color Picker widget
# color_picker = widgets.ColorPicker(
#     description="Pick a Color:",
#     value="#4682B4"  # Default color (Steel Blue)
# )

# # Output widget to display the HEX code
# output = widgets.Output()

# # Function to display the selected color HEX value
# def show_hex(change):
#     with output:
#         output.clear_output(wait=True)
#         print(f"Selected HEX Color: {change['new']}")

# # Attach observer to update HEX value dynamically
# color_picker.observe(show_hex, names="value")

# # Display the widgets
# display(color_picker, output)


In [8]:
# Create a button with a grey background
grey_button = widgets.Button(
    description="Refresh now",
    style={"button_color": "#ecf0f3"},  # Grey background
    layout=widgets.Layout(
        margin="10px"  # Adds 10px margin on all sides
        # width="120px",  # Optional: Set button width
        # height="40px"   # Optional: Set button height
    )
)

# # Display the button
# display(grey_button)


In [9]:
%%capture
%load_ext rpy2.ipython
%run -i ./MDT_Functions_copy.py

In [10]:
# 0.01### INPUT DATA
inputSpatial()

# SIMULATE RESPONSE
slider1_container, graph1 = simulateReponse()

# ## simulate resposne
# Javascript("Jupyter.notebook.execute_cells([2])")

C:\Users\alifr\anaconda3\envs\ericapp\Lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [11]:
graph1_container = widgets.HBox(
                        [graph1, checkbox_group],
                        layout=widgets.Layout(
                            border="2px solid black",  # 2px black border
                            margin = "2px 0px 0px 0px"
                        )
                    )

In [12]:
# display(graph1_container)

In [13]:
## SIMULATE OCCUANPCY DATA
slider2_container, graph2_container = simulateOccupancyData()

In [14]:
# display(graph2_container)

In [15]:
# import ipywidgets as widgets
# from IPython.display import display

# # Define widgets
# camConfig = widgets.Dropdown(
#     options=[(f"{i}", i) for i in range(1, 4)],
#     value=1,
#     description="camConfig:",
#     style={"description_width": "initial"}
# )
# siteScenN = widgets.IntSlider(min=1, max=50, step=1, value=1, description="siteScenN:")
# maxCam = widgets.IntSlider(min=1, max=200, step=1, value=1, description="maxCam:")
# minCam = widgets.IntSlider(min=1, max=200, step=1, value=1, description="minCam:")
# durScenN = widgets.IntSlider(min=1, max=50, step=1, value=1, description="durScenN:")
# maxDur = widgets.IntSlider(min=1, max=50, step=1, value=1, description="maxDur:")
# minDur = widgets.IntSlider(min=1, max=50, step=1, value=1, description="minDur:")

# # Output widget to display results dynamically
# output = widgets.Output()

# # Function to update dynamically
# def func_simulateOccupancyData(change=None):
#     with output:
#         output.clear_output(wait=True)  # Clear previous output
#         print(f"Simulating Occupancy Data with values:")
#         print(f"camConfig: {camConfig.value}")
#         print(f"siteScenN: {siteScenN.value}")
#         print(f"maxCam: {maxCam.value}")
#         print(f"minCam: {minCam.value}")
#         print(f"durScenN: {durScenN.value}")
#         print(f"maxDur: {maxDur.value}")
#         print(f"minDur: {minDur.value}")

# # Attach observe event to all widgets
# for widget in [camConfig, siteScenN, maxCam, minCam, durScenN, maxDur, minDur]:
#     widget.observe(func_simulateOccupancyData, names="value")

# # Display widgets and output area
# display(camConfig, siteScenN, maxCam, minCam, durScenN, maxDur, minDur, output)

# # Initial function call to show default values
# func_simulateOccupancyData()


In [16]:
# print(type(trueOcc))

In [17]:
# r_script = """
#         library(unmarked)
#         options(warn=2)  # Convert warnings to errors for debugging

#         getParams <- function(modObject) {
#             outParams <- list()
#             psiTab <- predict(modObject, type="state", newdata=data.frame(1))
#             outParams$psi <- psiTab$Predicted
#             outParams$psiSE <- psiTab$SE
#             outParams$psiBias <- outParams$psi - trueOcc

#             pTab <- predict(modObject, type="det", newdata=data.frame(1))
#             outParams$p <- pTab$Predicted
#             outParams$pSE <- pTab$SE
#             outParams$pBias <- outParams$p - meanDetection

#             return(outParams)
#         }

#         OccOutTab <- data.frame(CamN = NA, IntervalsN = NA, Response = NA, Estimate = NA,
#                                SE = NA, Bias = NA, inSig = NA)

#         dhScenDirs <- list.dirs(paste0(cwd, '/Data/DetectionHistories'), recursive = FALSE)

#         scenCount <- 0
#         FailCount <- 0 
#         for (dir in dhScenDirs) {
#             scenI <- unlist(strsplit(dir, "/"))
#             scen <- scenI[length(scenI)]
#             scen <- unlist(strsplit(scen, "_"))
#             CameraNumber <- scen[1]
#             VisitsNumber <- scen[2]
            
#             dhScens <- list.files(dir, pattern="\\\\.csv$")  # Correctly escaped backslash
        
#             for (scen in dhScens) {        
#                 dh <- read.csv(paste0(dir, "/", scen), header=TRUE)
            
#                 ## Fit model to scenario data
#                 umf <- unmarkedFrameOccu(y=as.matrix(dh))  # Organize data
#                 fm <- try(occu(~1 ~1, umf))  # Fit a model
#                 modOut <- try(coef(fm))  # Extract model coefficients
        
#                 if (class(modOut)[1] != "try-error") {    
#                     OP <- try(getParams(fm), silent=TRUE)
                    
#                     if (class(OP)[1] != "try-error") {
#                         scenCount <- scenCount + 1
#                         OccOutTab[scenCount, "CamN"] <- CameraNumber
#                         OccOutTab[scenCount, "IntervalsN"] <- VisitsNumber
#                         OccOutTab[scenCount, "Response"] <- "psi"
#                         OccOutTab[scenCount, "Estimate"] <- OP$psi
#                         OccOutTab[scenCount, "SE"] <- OP$psiSE
#                         OccOutTab[scenCount, "Bias"] <- OP$psiBias
#                         OccOutTab[scenCount, "inSig"] <- dplyr::between(0, OP$psi - (OP$psiSE * 2.58), OP$psi + (OP$psiSE * 2.58))
#                     }
#                 }
#             }
#         }

#         suppressPackageStartupMessages(library(tidyverse))
#         options(warn=-1)  # Suppress warnings
        
#         # Ensure OccOutTab exists and is not empty before filtering
#         if (exists("OccOutTab") && nrow(OccOutTab) > 0) {
#             SumTab <- OccOutTab %>%
#                 filter(!is.na(inSig) & inSig) %>%  # Ensure 'inSig' is not NA before filtering
#                 group_by(CamN, IntervalsN, Response) %>%
#                 summarise(
#                     N = n(),  # More reliable than length(Estimate)
#                     meanEstimate = mean(Estimate, na.rm=TRUE),
#                     sdEstimate = sd(Estimate, na.rm=TRUE),
#                     seEstimate = sdEstimate / sqrt(N),
#                     UpperCI_Est = quantile(Estimate, probs=0.9, na.rm=TRUE),
#                     LowerCI_Est = quantile(Estimate, probs=0.1, na.rm=TRUE),
#                     meanBias = mean(Bias, na.rm=TRUE),
#                     sdBias = sd(Bias, na.rm=TRUE),
#                     seBias = sdBias / sqrt(N),
#                     UpperCI_Bias = quantile(Bias, probs=0.9, na.rm=TRUE),
#                     LowerCI_Bias = quantile(Bias, probs=0.1, na.rm=TRUE),
#                     .groups = "drop"  # Prevent grouped output warning
#                 ) %>%
#                 mutate(
#                     CamN = as.factor(CamN),
#                     Durations = as.numeric(IntervalsN)
#                 )
#         } else {
#             SumTab <- data.frame()  # Return an empty dataframe if OccOutTab does not exist or is empty
#             message("Warning: OccOutTab is empty or does not exist. SumTab is empty.")
#         }


#         library(ggplot2)
#         library(dplyr)
        
#         pd <- position_dodge(0.5)
        
#         # Generate the plot
#         plot <- SumTab %>%
#             filter(!is.na(sdEstimate), meanEstimate < 0.99, meanEstimate > 0.001) %>%
#             ggplot(
#                 aes(x=Durations, y=meanBias, ymin=LowerCI_Bias, ymax=UpperCI_Bias, colour=CamN)
#             ) + 
#                 geom_errorbar(width=0.1, position=pd) +
#                 geom_point(size=3, position=pd) +        
#                 facet_grid(Response~., scales="free") +
#                 geom_abline(size=0.5, intercept=0, slope=0)
        
#         # Save the plot as an image file
#         plot_path <- "my_plot.png"  # Saves in the current working directory
#         ggsave(plot_path, plot, width=6, height=4, dpi=100)

# """


# # Run the formatted R script
# ro.r(r_script)

In [18]:
# ####---working 
#         library(unmarked)
#         options(warn=2)  # Convert warnings to errors for debugging

#         getParams <- function(modObject) {
#             outParams <- list()
#             psiTab <- predict(modObject, type="state", newdata=data.frame(1))
#             outParams$psi <- psiTab$Predicted
#             outParams$psiSE <- psiTab$SE
#             outParams$psiBias <- outParams$psi - trueOcc

#             pTab <- predict(modObject, type="det", newdata=data.frame(1))
#             outParams$p <- pTab$Predicted
#             outParams$pSE <- pTab$SE
#             outParams$pBias <- outParams$p - meanDetection

#             return(outParams)
#         }

#         OccOutTab <- data.frame(CamN = NA, IntervalsN = NA, Response = NA, Estimate = NA,
#                                SE = NA, Bias = NA, inSig = NA)

#         dhScenDirs <- list.dirs(paste0(cwd, '/Data/DetectionHistories'), recursive = FALSE)

#         scenCount <- 0
#         FailCount <- 0 

In [19]:
####-----------copied to .py file

In [20]:
# %R -i cwd
# %R -i trueOcc
# %R -i popPX
# %R -i N
# %R -i meanDetection

In [21]:
# %%R

# library(unmarked)
# options(warn=2)

# getParams = function(modObject){
    
#     outParams = list()
#     psiTab = predict(modObject, type="state",newdata=data.frame(1))
#     outParams$psi = psiTab$Predicted
#     outParams$psiSE = psiTab$SE
#     outParams$psiBias = outParams$psi - trueOcc
    
#     pTab = predict(modObject, type="det",newdata=data.frame(1))
#     outParams$p = pTab$Predicted
#     outParams$pSE = pTab$SE
#     outParams$pBias = outParams$p - meanDetection
    
#     return(outParams)
#     }


# OccOutTab = data.frame(CamN = NA, IntervalsN = NA, Response = NA, Estimate = NA,
#                        SE = NA, Bias = NA, inSig = NA)


# dhScenDirs = list.dirs(paste0(cwd,'/Data/DetectionHistories'),recursive = FALSE)  
# # print(dhScenDirs)
# scenCount = 0
# FailCount = 0 

# for (dir in dhScenDirs){    

#     scenI = unlist(strsplit(dir,"/"))
#     scen = scenI[length(scenI)]
#     scen = unlist(strsplit(scen,"_"))
#     CameraNumber = scen[1]
#     VisitsNumber = scen[2]

#     dhScens = list.files(dir,pattern="\\.csv$")
    
#     for (scen in dhScens){        
#         dh = read.csv(paste0(dir,"/",scen),header=T)

#         ## fit model to scenario data
#         umf <- unmarkedFrameOccu(y=as.matrix(dh)) # organize data
#         fm <- try(occu(~1 ~1, umf)) # fit a model
#         modOut = try(coef(fm)) 
# #         print('')
# #         print('')
# #         print(class(fm))
        
#         if(class(modOut) != "try-error" ){    

# #             ## model outputs
# #             psiTab = predict(fm, type="state",newdata=data.frame(1))
# #             psi = psiTab$Predicted
# #             psiSE = psiTab$SE
# #             psiBias = psi - trueOcc
# #             pTab = predict(fm, type="det",newdata=data.frame(1))
# #             p = pTab$Predicted
# #             pSE = pTab$SE
# #             pBias = p - meanDetection
            
#             OP = try(getParams(fm))
#             if(class(OP)!= "try-error"){

#                 ## occupancy info
#                 scenCount = scenCount + 1
#                 OccOutTab[scenCount,"CamN"] = CameraNumber
#                 OccOutTab[scenCount,"IntervalsN"] = VisitsNumber
#                 OccOutTab[scenCount,"Response"] = "psi"
#                 OccOutTab[scenCount,"Estimate"] = OP$psi
#                 OccOutTab[scenCount,"SE"] = OP$psiSE
#                 OccOutTab[scenCount,"Bias"] = OP$psiBias
#                 OccOutTab[scenCount,"inSig"] = dplyr::between(0,OP$psi-(OP$psiSE*2.58),OP$psi+(OP$psiSE*2.58))

#                 ## detection info
#                 scenCount = scenCount + 1
#                 OccOutTab[scenCount,"CamN"] = CameraNumber
#                 OccOutTab[scenCount,"IntervalsN"] = VisitsNumber
#                 OccOutTab[scenCount,"Response"] = "p"
#                 OccOutTab[scenCount,"Estimate"] = OP$p
#                 OccOutTab[scenCount,"SE"] = OP$pSE
#                 OccOutTab[scenCount,"Bias"] = OP$pBias
#                 OccOutTab[scenCount,"inSig"] = dplyr::between(0,OP$p-(OP$pSE*2.58),OP$p+(OP$pSE*2.58))
#             }
#             else{
#                 FailCount = FailCount + 1 
#                 print("Model failed")
#             }
#         }
#         else{
#             FailCount = FailCount + 1 
#             print("Model failed")
#         }
#     }
# }



# # print("Fit occupancy models to all detection histories.")
# # print(paste0(FailCount," models failed."))

In [22]:
# %%R
# # Suppress package loading messages and warnings
# suppressPackageStartupMessages(library(tidyverse))
# options(warn=-1)  # Suppress warnings

# # Summarization process
# SumTab = OccOutTab %>%
#     filter(!inSig) %>%
#     group_by(CamN, IntervalsN, Response) %>%
#     summarise(
#         N = length(Estimate),
        
#         meanEstimate = mean(Estimate),
#         sdEstimate = sd(Estimate),
#         seEstimate   = sdEstimate / sqrt(N),
#         UpperCI_Est = quantile(Estimate, probs = 0.9),
#         LowerCI_Est = quantile(Estimate, probs = 0.1),
        
#         meanSE = mean(SE),
#         sdSE = sd(SE),
#         seSE   = sdSE / sqrt(N),
#         UpperCI_SE = quantile(SE, probs = 0.9),
#         LowerCI_SE = quantile(SE, probs = 0.1),
        
#         meanBias = mean(Bias),
#         sdBias = sd(Bias),
#         seBias   = sdBias / sqrt(N),
#         UpperCI_Bias = quantile(Bias, probs = 0.9),
#         LowerCI_Bias = quantile(Bias, probs = 0.1),
#         .groups = "drop"  # Prevent "grouped output" message
#     ) %>%
#     mutate(CamN = as.factor(CamN),
#            Durations = as.numeric(IntervalsN))



# # print(SumTab)
# # print(meanDetection)

In [23]:
# %%R
# # print(meanEstimate)
# pd <- position_dodge(0.5)
# SumTab %>%
#     filter(!is.na(sdEstimate),meanEstimate < 0.99, meanEstimate > 0.001)%>%
#     ggplot(
#         aes(x=Durations, y=meanBias,ymin=LowerCI_Bias, ymax=UpperCI_Bias, colour=CamN)) + 
#         #aes(x=Durations, y=meanEstimate,ymin=UpperCI_Est, ymax=LowerCI_Est, colour=CamN)) +
#         geom_errorbar(width=0.1,position = pd)+
#         geom_point(size = 3, position = pd) +        
#         facet_grid(Response~.,scales="free") +
#         geom_abline(size=0.5,intercept = 0, slope=0) 

In [24]:
# %%R -o plot_path

# library(ggplot2)
# library(dplyr)

# pd <- position_dodge(0.5)

# # Generate the plot
# plot <- SumTab %>%
#     filter(!is.na(sdEstimate), meanEstimate < 0.99, meanEstimate > 0.001) %>%
#     ggplot(
#         aes(x=Durations, y=meanBias, ymin=LowerCI_Bias, ymax=UpperCI_Bias, colour=CamN)
#     ) + 
#         geom_errorbar(width=0.1, position=pd) +
#         geom_point(size=3, position=pd) +        
#         facet_grid(Response~., scales="free") +
#         geom_abline(size=0.5, intercept=0, slope=0)

# # Save the plot as an image file
# plot_path <- "my_plot.png"  # Saves it in the current working directory
# ggsave(plot_path, plot, width=6, height=4, dpi=100)


In [25]:
# # ## -- REDUNDANT-------------DELETE after checking

# import ipywidgets as widgets
# from IPython.display import display, Image

# # Convert R's StrVector to a regular Python string
# plot_path = str(plot_path[0])  # Extract the string value

# # # Ensure the path is correct
# # print("Plot saved at:", plot_path)  # Debugging step

# # Open and display the saved plot using ipywidgets
# with open(plot_path, "rb") as f:
#     plot_widget = widgets.Image(value=f.read(), format='png')

# # Wrap it inside a VBox container
# graph2_first_container = widgets.VBox([plot_widget],
#                         layout=widgets.Layout(
#                             width = "auto"
#                         ))

# # Wrap it inside a VBox container
# graph2_container = widgets.VBox([graph2_first_container],
#                         layout=widgets.Layout(
#                             width = "100%",
#                             flex_flow="column",
#                             align_items="center",
#                             border="2px solid black"  # 2px black border
#                         ))

# # # Display the VBox containing the plot
# # display(vbox_container)

In [26]:
# ### IMPORTANT---------------Needed for dynamic update -------------

# import ipywidgets as widgets
# from IPython.display import display, clear_output
# import rpy2.robjects as ro
# from rpy2.robjects.packages import importr
# import rpy2.robjects.pandas2ri

# # Activate automatic conversion between pandas and R dataframes
# rpy2.robjects.pandas2ri.activate()

# # Define UI widgets for input adjustments
# mean_estimate_slider = widgets.FloatRangeSlider(
#     value=[0.001, 0.99], min=0.0, max=1.0, step=0.01,
#     description="Mean Estimate",
#     continuous_update=False
# )

# # Output widget for displaying the plot
# plot_output = widgets.Output()

# # Function to update the plot dynamically
# def update_plot(change):
#     with plot_output:
#         clear_output(wait=True)  # Clear previous output before updating

#         # Convert Python slider values to R
#         min_estimate, max_estimate = mean_estimate_slider.value
#         ro.r.assign("min_estimate", min_estimate)
#         ro.r.assign("max_estimate", max_estimate)

#         # Execute R code to generate the plot
#         ro.r('''
#             library(ggplot2)
#             library(dplyr)
#             pd <- position_dodge(0.5)

#             # Filter data dynamically
#             print(min_estimate)
#             plot <- SumTab %>%
#                 filter(!is.na(sdEstimate), meanEstimate < max_estimate, meanEstimate > min_estimate) %>%
#                 ggplot(
#                     aes(x=Durations, y=meanBias, ymin=LowerCI_Bias, ymax=UpperCI_Bias, colour=CamN)
#                 ) + 
#                     geom_errorbar(width=0.1, position=pd) +
#                     geom_point(size=3, position=pd) +        
#                     facet_grid(Response~., scales="free") +
#                     geom_abline(size=0.5, intercept=0, slope=0)

#             # Save plot to a temporary file
#             plot_path <- "my_plot.png"
#             print(plot_path)
#             ggsave(plot_path, plot, width=6, height=4, dpi=100)
#         ''')

#         # Retrieve the saved plot path
#         plot_path = str(ro.r('plot_path')[0])  # Convert R StrVector to Python string

#         # Display updated plot
#         with open(plot_path, "rb") as f:
#             img_widget = widgets.Image(value=f.read(), format='png')

#         display(img_widget)

# # Attach the update function to the slider
# mean_estimate_slider.observe(update_plot, names='value')

# # Wrap it inside a VBox container
# boxPlot_container = widgets.VBox([mean_estimate_slider, plot_output])

# # Display the VBox containing the plot
# display(boxPlot_container)

# # Generate the initial plot
# update_plot(None)


In [27]:
slider_Vbox = widgets.VBox([add_info_widget, slider1_container, slider2_container, grey_button],
    layout=widgets.Layout(width="30%"))

graph_Vbox = widgets.VBox(
    [graph1_container, graph2_container],
    layout=widgets.Layout(
        width="70%",
        display="flex",
        flex_flow="column",
        align_items="center",  # Optional: Aligns items in center
        grid_gap="5px"  # Adds 10px spacing between elements
    )
)
slide_graphHbox = widgets.HBox([slider_Vbox, graph_Vbox],
    layout=widgets.Layout(width="100%"))
main_container = widgets.VBox(
    [title_widget, slide_graphHbox],
    layout=widgets.Layout(
        width="100%"    
    )
)

In [28]:
display(main_container)

In [31]:
import ipywidgets as widgets
from IPython.display import display
import base64

# Define file path
file_path = "r_plot.png"

# Function to generate the download link
def get_download_link(file_path):
    with open(file_path, "rb") as file:
        file_data = file.read()
    file_base64 = base64.b64encode(file_data).decode()  # Encode file to Base64

    download_html = f'''
    <a download="{file_path}" href="data:image/png;base64,{file_base64}" target="_blank">
        <button style="padding:10px; font-size:14px; background-color:#4CAF50; color:white; border:none; border-radius:5px; cursor:pointer;">
            📥 Download Plot
        </button>
    </a>
    '''
    return download_html

# Create button widget
download_button = widgets.HTML(value=get_download_link(file_path))

# Display the button
display(download_button)


HTML(value='\n    <a download="r_plot.png" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAlgAAAGQCAIAAAD…

In [55]:
# import ipywidgets as widgets
# from IPython.display import display

# # Define widgets
# camConfig = widgets.Dropdown(
#     options=[(f"{i}", i) for i in range(1, 4)],  # Dropdown options
#     value=1,  # Default selected value
#     description="camConfig:",
#     style={"description_width": "initial"}  # Ensures label is fully visible
# )
# siteScenN = widgets.IntSlider(min=1, max=50, step=1, value=0, description="siteScenN:")
# maxCam = widgets.IntSlider(min=1, max=200, step=1, value=0, description="maxCam:")
# minCam = widgets.IntSlider(min=1, max=200, step=1, value=0, description="minCam:")
# durScenN = widgets.IntSlider(min=1, max=50, step=1, value=0, description="durScenN:")
# maxDur = widgets.IntSlider(min=1, max=50, step=1, value=0, description="maxDur:")
# minDur = widgets.IntSlider(min=1, max=50, step=1, value=0, description="minDur:")

# # Output widget to display results dynamically
# simulateOccupancyData_output = widgets.Output()

# # Function to display simulation results dynamically
# def func_simulateOccupancyData(change=None):
#     with simulateOccupancyData_output:
#         simulateOccupancyData_output.clear_output(wait=True)  # Clear previous output
#         print(f"Simulating Occupancy Data with values:")
#         print(f"camConfig: {camConfig.value}")
#         print(f"siteScenN: {siteScenN.value}")
#         print(f"maxCam: {maxCam.value}")
#         print(f"minCam: {minCam.value}")
#         print(f"durScenN: {durScenN.value}")
#         print(f"maxDur: {maxDur.value}")
#         print(f"minDur: {minDur.value}")

# # Function to observe changes and return a VBox
# def simulateOccupancyData():
#     # Attach observer to all widgets
#     for widget in [camConfig, siteScenN, maxCam, minCam, durScenN, maxDur, minDur]:
#         widget.observe(func_simulateOccupancyData, names="value")

#     # Create VBox layout
#     simulateOccupancyData_Vbox = widgets.VBox(
#         [camConfig, siteScenN, maxCam, minCam, durScenN, maxDur, minDur, simulateOccupancyData_output],
#         layout=widgets.Layout(width='auto')
#     )
    
#     # Call function initially to show default values
#     func_simulateOccupancyData()

#     return simulateOccupancyData_Vbox

# # Display the widget
# display(simulateOccupancyData())
